In [2]:
import pandas as pd

origin_data1_31 = pd.read_csv('data2/CGM1_dSr31.csv', encoding='cp949')
origin_data2_31 = pd.read_csv('data2/CGM2_dSr31.csv', encoding='cp949')
origin_data3_31 = pd.read_csv('data2/CGM3_dSr31.csv', encoding='cp949')
origin_data4_31 = pd.read_csv('data2/CGM4_dSr31.csv', encoding='cp949')

In [3]:
def filter_data(origin_data, columns):
    data = origin_data[columns]
    data = data[data["Glu2(mg/dl)"] > 0]
    data = data[data[columns[3]] != 0]
    return data

common_var = ["Glu2(mg/dl)", "LD1 Temp", "LD2 Temp", "LD3 Temp", "Rx1 Temp", "Rx2 Temp", "S1 T(C)", "S2 T(C)", "S3 T(C)", "FR Mon", "LD_Bias_Av"]
columns31 = common_var + ["mPDdSr31", "T-rPDdSr31", "R-rPDdSr31"]

data_R1_31 = filter_data(origin_data1_31, columns31)
data_R2_31 = filter_data(origin_data2_31, columns31)
data_R3_31 = filter_data(origin_data3_31, columns31)
data_R4_31 = filter_data(origin_data4_31, columns31)

data31 = pd.concat([data_R1_31, data_R2_31, data_R4_31])
print(len(data31))

744


In [8]:
import torch
from pickle import dump
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def preprocess(data , dt=False) -> tuple:
    if dt:
        data = data.astype({'Glu2(mg/dl)':'int'})
    
    X = data.drop('Glu2(mg/dl)', axis=1).values
    y = data['Glu2(mg/dl)'].values
    y = y.round(0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    dump(scaler, open('./scaler31.pkl', 'wb'))
    
    return torch.tensor(X_train, dtype=torch.float32), \
        torch.tensor(y_train, dtype=torch.float32), \
        torch.tensor(X_test, dtype=torch.float32), \
        torch.tensor(y_test, dtype=torch.float32)

In [9]:
data31 = preprocess(data31, True)